In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/SRR8608947.assembly.len10.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/RedKingCrab.len10.pkl"

In [3]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [4]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,QGDMVVIQCLVHKGKHSLCHLLGAVKVMVTVGKN,34
1,1,VLPDGDHDLDRAQKVTETVLAFVYKALNDHHVSLRVPCSNPT,42
2,2,MYGSTGPPHSEHLGGRGGCSLA,24
3,3,LLWVVVVFWMLEVVSLVSVVSVVDMVLVLESVSVVDSVVVLE,42
4,4,FNLRSNLTSFLILSFGPALCELFWETQWSGSTNLEANSN,40
...,...,...,...
6492118,6492118,MGCKFPHLFPVNSPQMVKISAQ,24
6492119,6492119,MSYINRKCPLKSNFEIKYLWKCTCAF,28
6492120,6492120,MLYPKLLFLQ,12
6492121,6492121,KDGVRKYMNEFVAGWEEGRRPTGDRGAWGRFVPGHPFGDNGVR,43


In [ ]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

Remove Duplicate

In [ ]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

In [ ]:
df.to_pickle(plk)
tmp_df = pd.read_pickle(plk)
tmp_df